In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt


df = pd.read_csv('2020.csv')
print(df.head())
print(df.info())


      FL_DATE OP_CARRIER  OP_CARRIER_FL_NUM ORIGIN DEST  CRS_DEP_TIME  \
0  2020-01-01         WN               5888    ONT  SFO        1851.0   
1  2020-01-01         WN               6276    ONT  SFO        1146.0   
2  2020-01-01         WN               4598    ONT  SJC        2016.0   
3  2020-01-01         WN               4761    ONT  SJC        1350.0   
4  2020-01-01         WN               5162    ONT  SJC         916.0   

   DEP_TIME  DEP_DELAY  TAXI_OUT  WHEELS_OFF  ...  CRS_ELAPSED_TIME  \
0      41.0       44.0    1935.0      2049.0  ...               0.0   
1      -4.0       13.0    1159.0      1310.0  ...               NaN   
2      -4.0        8.0    2024.0      2121.0  ...               NaN   
3      10.0       10.0    1400.0      1503.0  ...               NaN   
4       1.0        6.0     922.0      1019.0  ...               NaN   

   ACTUAL_ELAPSED_TIME  AIR_TIME  DISTANCE  CARRIER_DELAY  WEATHER_DELAY  \
0                 33.0       NaN       NaN            NaN 